In [1]:
import sys
sys.path.append("../module/")
sys.path.append("../learning/")

import pickle
import menconn
from typelinking import *
import time
import marisa_trie
import pickle
from dataset import *
from wikidata_linker_utils.offset_array import OffsetArray
import train_type as tp
from collections import defaultdict
from tqdm import tqdm
import re
from functools import partial
from tqdm import tqdm_notebook
import os

../module/typelinking.py:106: SyntaxWarning: name 'dataroot' is assigned to before global declaration
  global dataroot
/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
with open("test_data.pkl", 'rb') as f:
    data = pickle.load(f)

In [3]:
results = []
dataroot = '..'
tagger = tp.SequenceTagger(os.path.join(dataroot,'my_great_model/'))
type_oracle = load_oracle_classification(os.path.join(dataroot, "data/classifications/type_classification"))
trie_index2indices_values, trie_index2indices_counts, trie = load_trie(os.path.join(dataroot, 'data/en_trie'))
with open(os.path.join(dataroot, 'data/wikidata/indicies2wikititle.pkl'), 'rb') as hdl:
    indices2title = pickle.load(hdl)

# 1. LinkProbだけで実行

In [7]:
for d in tqdm_notebook(data):
    if len(d[2]) == 0:
        continue
    sentence = d[1]
    ts = [re.sub(r'[^\w ]','', str(t[0])).lower() for t in d[2]]
    ts = [re.sub(r'[ ]+', ' ', t) for t in ts]
    true_entities = [str(t[1]).replace('_', ' ') for t in d[2]]
    entities = run(ts, None, None, indices2title, None, trie, trie_index2indices_values, trie_index2indices_counts, only_link=True)
    preds = []
    for entity in entities:
        if entity is not None:
            preds.append(entity['en'])
        else:
            preds.append(None)
    results += [{'doc_id':d[0], 'mention':x, 'true': y, 'pred': z} for x,y,z in zip(ts, true_entities, preds)]

HBox(children=(IntProgress(value=0, max=1392), HTML(value='')))

In [8]:
import pandas as pd
df = pd.DataFrame(results)

In [9]:
matched = df['pred'] == df['true']
length = df['pred'].shape[0]
assert len(df['pred']) == len(df['true'])

accuracy = float(sum(matched))/float(length)
accuracy

0.6749112999974475

In [11]:
from sklearn.metrics import f1_score
true = df['true'] == df['true']
pred = matched
f1_score(true, pred)

0.8059069157853027

# LinkProb + TypeProbで実行

In [19]:
for d in tqdm_notebook(data):
    if len(d[2]) == 0:
        continue
    sentence = d[1]
    ts = [re.sub(r'[^\w ]','', str(t[0])).lower() for t in d[2]]
    ts = [re.sub(r'[ ]+', ' ', t) for t in ts]
    true_entities = [str(t[1]).replace('_', ' ') for t in d[2]]
    tokenize = partial(menconn.en_tokenize, ts=ts)
    sent_splits, model_probs = solve_model_probs(sentence, tagger, tokenize=tokenize)
    entities = run(ts, sent_splits, model_probs, indices2title, type_oracle, trie, trie_index2indices_values, trie_index2indices_counts)
    preds = []
    for entity in entities:
        if entity is not None:
            preds.append(entity['en'])
        else:
            preds.append(None)
    results += [{'doc_id':d[0], 'mention':x, 'true': y, 'pred': z} for x,y,z in zip(ts, true_entities, preds)]

HBox(children=(IntProgress(value=0, max=1392), HTML(value='')))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.5/dist-packages/tqdm/_monitor.py", line 63, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600


In [27]:
import pandas as pd
df = pd.DataFrame(results)

In [28]:
df.to_csv("results.csv", index=False)

In [37]:
matched = df['pred'] == df['true']
length = df['pred'].shape[0]
assert len(df['pred']) == len(df['true'])

accuracy = float(sum(matched))/float(length)
accuracy

0.6836213348953757

In [41]:
from sklearn.metrics import f1_score
true = df['true'] == df['true']
pred = matched
f1_score(true, pred)

0.812084428637699

In [42]:
df

,doc_id,mention,pred,true
0,1,bse,Bovine spongiform encephalopathy,Bovine spongiform encephalopathy
1,1,european union,European Union,European Union
2,1,germany,Germany,Germany
3,1,britain,United Kingdom,United Kingdom
4,1,commission,Ship commissioning,European Commission
5,1,franz fischler,Franz Fischler,Franz Fischler
6,1,european commission,European Commission,European Commission
7,1,france,France,France
8,1,spanish,Spanish language,Spain
9,1,europe,Europe,Europe
